In [2]:
import json

In [5]:
def load_and_check(path):
    data = json.load(open(path))

    s = ""
    running_mean = 0

    for key in data:
        metric_acc = data[key]["acc"]*100
        s += f"{round(metric_acc, 1)} & "
        running_mean += metric_acc
    
    running_mean /= len(data)
    s += f"{round(running_mean, 2)}"

    print(s)

In [6]:
load_and_check("/data/lliu/huffman/models/meta-llama/Meta-Llama-3-8B/compressed/167/results.json")

67.2 & 56.7 & 74.3 & 69.7 & 36.0 & 60.76


In [7]:
load_and_check("/data/lliu/huffman/models/meta-llama/Llama-2-13b-hf/compressed/166/results.json")

68.8 & 64.3 & 76.2 & 72.9 & 41.0 & 64.66


In [8]:
load_and_check("/data/lliu/huffman/models/meta-llama/Llama-2-7b-hf/compressed/166/results.json")

64.9 & 54.5 & 73.0 & 66.8 & 33.6 & 58.57
